<a href="https://colab.research.google.com/github/NY-KO/d2l-notebooks/blob/main/%E5%8A%A8%E6%89%8B%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#引言

训练过程通常包含如下步骤：
1. 从一个随机初始化参数的模型开始，这个模型基本没有“智能”；
2. 获取一些数据样本（例如，音频片段以及对应的是或否标签）；
3. 调整参数，使模型在这些样本中表现得更好；
4. 重复第（2）步和第（3）步，直到模型在任务中的表现令人满意。

#预备知识

##2.1.1入门
张量表示一个由数值组成的数组，这个数组可能有多个维度。具有一个轴的张量对应数学上的向量（vector）；
具有两个轴的张量对应数学上的矩阵（matrix）；具有两个轴以上的张量没有特殊的数学名称。  
*生成矩阵逻辑：先torch.arange生成行向量或列向量，再通过reshape生成矩阵（全是0或1的矩阵例外），直接用torch.tensor生成矩阵也行

In [2]:
import torch

In [3]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [17]:
#补充
# torch.arange(start, end, step)
x1 = torch.arange(0, 12, 1)
x1

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [4]:
x.shape

torch.Size([12])

In [5]:
x.numel()

12

In [6]:
#把张量x从形状为（12,）的行向量转换为形状为（3,4）的矩阵
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [7]:
#补充
#我们可以通过-1来调用此自动计算出维度的功能。即我们可以用x.reshape(-1,4)或x.reshape(3,-1)来取代x.reshape(3,4)
X = x.reshape(3, -1)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

⬆️2025.5.19

In [11]:
#补充
#张量中所有元素的总个数
X.numel()

12

In [12]:
#补充
#返回一个 torch.Size 对象，是一个元组，表示张量在各个维度上的长度
X.shape

torch.Size([3, 4])

In [18]:
#torch.zeros(2, 3, 4)分开传3个 int ，torch.zeros((2, 3, 4))一起传一个tuple(2, 3, 4)，其实等价
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [20]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [21]:
#每个元素都从均值为0、标准差为1的标准高斯分布（正态分布）中随机采样。
torch.randn(2, 3, 4)

tensor([[[-1.2330,  1.8309,  0.4511, -0.0202],
         [-0.7316,  0.3561, -0.3079, -1.0855],
         [-0.1740, -1.2044,  1.1922, -0.9498]],

        [[ 0.7453,  0.3535,  1.0992, -0.0221],
         [-0.2406,  0.0058, -1.8218,  1.6592],
         [ 0.2733,  0.6519,  0.5134,  0.9819]]])

In [24]:
torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

##2.1.2运算符  
按元素（elementwise）运算:它们将标准标量运算符应用于数组的每个元素。

In [25]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x / y, x * y, x ** y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([ 1.,  4., 16., 64.]))

In [26]:
#指数函数计算
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

In [27]:
#沿行（轴0),按列（轴1)连结两个矩阵
X = torch.arange(12, dtype = torch.float32).reshape(3, 4)
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim = 0), torch.cat((X, Y), dim = 1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))